# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yougui\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yougui\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yougui\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)  
X = df['message']
Y = df.drop(columns=['id','message','original','genre'])
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    raw_toks = word_tokenize(text)
    lem = WordNetLemmatizer()
    tokens = []
    for tok in raw_toks:
        tokens.append(lem.lemmatize(tok).strip().lower())        
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [16]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)), ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(RandomForestClassifier())) ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = 0.3, random_state = 999)
pipeline.fit(X_train, y_train)

C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:2

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [18]:
y_pred = pipeline.predict(X_test)

In [19]:
y_pred.shape

(7865, 36)

In [20]:
y_pred = pipeline.predict(X_test)
for i, col in enumerate(y_train):
    print('............................')
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))

............................
related
              precision    recall  f1-score   support

           0       0.62      0.36      0.45      1854
           1       0.82      0.93      0.87      5952
           2       0.46      0.10      0.17        59

   micro avg       0.79      0.79      0.79      7865
   macro avg       0.63      0.46      0.50      7865
weighted avg       0.77      0.79      0.77      7865

............................
request
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      6506
           1       0.81      0.41      0.55      1359

   micro avg       0.88      0.88      0.88      7865
   macro avg       0.85      0.70      0.74      7865
weighted avg       0.88      0.88      0.87      7865

............................
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7828
           1       0.00      0.00      0.00        37

   micro avg       

C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



............................
missing_people
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7777
           1       1.00      0.01      0.02        88

   micro avg       0.99      0.99      0.99      7865
   macro avg       0.99      0.51      0.51      7865
weighted avg       0.99      0.99      0.98      7865

............................
refugees
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      7592
           1       0.00      0.00      0.00       273

   micro avg       0.97      0.97      0.97      7865
   macro avg       0.48      0.50      0.49      7865
weighted avg       0.93      0.97      0.95      7865

............................
death
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      7513
           1       0.82      0.12      0.21       352

   micro avg       0.96      0.96      0.96      7865
   macro a

### 6. Improve your model
Use grid search to find better parameters. 

In [24]:
parameters = { 'clf__estimator__n_estimators': [50, 100,150], 
               'clf__estimator__max_depth': [5,10,20]} 

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=10)
cv.fit(X_train, y_train) 

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'clf__estimator__n_estimators': [20, 40, 80], 'clf__estimator__max_depth': [10, 20, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [25]:
cv.best_params_

{'clf__estimator__max_depth': 10, 'clf__estimator__n_estimators': 80}

In [30]:
y_pred = cv.predict(X_test)
print(y_pred.shape)
for i, col in enumerate(y_train):
    print('............................')
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))

(7865, 36)
............................
related
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1854
           1       0.76      1.00      0.86      5952
           2       0.00      0.00      0.00        59

   micro avg       0.76      0.76      0.76      7865
   macro avg       0.25      0.33      0.29      7865
weighted avg       0.57      0.76      0.65      7865

............................
request
              precision    recall  f1-score   support

           0       0.83      1.00      0.91      6506
           1       0.00      0.00      0.00      1359

   micro avg       0.83      0.83      0.83      7865
   macro avg       0.41      0.50      0.45      7865
weighted avg       0.68      0.83      0.75      7865

............................
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7828
           1       0.00      0.00      0.00        37

   micro

C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



............................
tools
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      7811
           1       0.00      0.00      0.00        54

   micro avg       0.99      0.99      0.99      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.99      0.99      0.99      7865

............................
hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7784
           1       0.00      0.00      0.00        81

   micro avg       0.99      0.99      0.99      7865
   macro avg       0.49      0.50      0.50      7865
weighted avg       0.98      0.99      0.98      7865

............................
shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7839
           1       0.00      0.00      0.00        26

   micro avg       1.00      1.00      1.00      7865
   macro avg      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [36]:
from sklearn.ensemble import    AdaBoostClassifier
ada_forest = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)), ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(AdaBoostClassifier(base_estimator=RandomForestClassifier(),n_estimators=100, learning_rate=0.9,
                        random_state=None ))) ])

In [37]:
ada_forest.fit(X_train, y_train) 

C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:2

C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:2

C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:2

C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:2

C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:2

...

C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:2

C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:2

C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:2

C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\yougui\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:2

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...=False),
          learning_rate=0.9, n_estimators=80, random_state=None),
           n_jobs=None))])

In [38]:
y_pred = ada_forest.predict(X_test)
print(y_pred.shape)
for i, col in enumerate(y_train):
    print('............................')
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))

(7865, 36)
............................
related
              precision    recall  f1-score   support

           0       0.69      0.30      0.42      1854
           1       0.81      0.96      0.88      5952
           2       0.69      0.15      0.25        59

   micro avg       0.80      0.80      0.80      7865
   macro avg       0.73      0.47      0.52      7865
weighted avg       0.78      0.80      0.77      7865

............................
request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      6506
           1       0.81      0.52      0.64      1359

   micro avg       0.90      0.90      0.90      7865
   macro avg       0.86      0.75      0.79      7865
weighted avg       0.89      0.90      0.89      7865

............................
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7828
           1       0.00      0.00      0.00        37

   micro


............................
hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7784
           1       0.00      0.00      0.00        81

   micro avg       0.99      0.99      0.99      7865
   macro avg       0.49      0.50      0.50      7865
weighted avg       0.98      0.99      0.98      7865

............................
shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7839
           1       0.00      0.00      0.00        26

   micro avg       1.00      1.00      1.00      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.99      1.00      0.99      7865

............................
aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7772
           1       0.11      0.02      0.04        93

   micro avg       0.99      0.99      0.99      7865
   macro avg

### 9. Export your model as a pickle file

In [40]:
import pickle 
with open('classifier.pkl', 'wb') as file:
    pickle.dump(ada_forest, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.